In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib
import h5py
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
from diffpy.pdfgetx.pdfconfig import PDFConfig
from diffpy.pdfgetx.pdfgetter import PDFGetter
from diffpy.morph.morphpy import morph_arrays

%matplotlib widget
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger("diffpy.pdfgetx").setLevel(logging.ERROR)
logging.getLogger("diffpy.pdfgetx.user").setLevel(logging.ERROR)
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
run_number = 35
sample_name = 'NSPS'
sample_composition = 'Na11SnPS12'
target_id = 0 #index of the delay_time Iq_off to use as target for morphing as well as calculate adhoc corrections for g(r)
q_min =8 #qmin for figure of merit
q_max = 9 #qmax for figure of merit
q_min_morph = 0 #qmin for normalization. None uses the smaller value. Not working for new diffpy.morph! Also is called xmin now.
q_max_morph = 12 #qmax for normalization. None uses the largest value. Not working for new diffpy.morph! Also is called xmax now.
scale = 1.01 
stretch = None
smear = None
baselineslope = None
qdamp = None
params = {'xmin': q_min_morph, 'xmax': q_max_morph, 'scale': scale,'stretch': stretch, 'smear':smear}
t0 = 0
points_away_t0_plot_on_off = 0

fit_qmin = 0.1
fit_qmax = 10
r_min_fom = 2
r_max_fom = 5
initial_guesses = {'rpoly': 1, 'qmin': 0.1, 'bgscale': 1}
squeeze = [0,0,0,0,0,0]

In [ ]:
cwd = Path().cwd()
input_path = cwd.parent/'MFX_hops'/'mfxl1044925'/'results'/'example_h5_files'/'smd_pyfai'
str_run_number = str(run_number).zfill(4)
delay_positions_path = input_path/f'mfx101343025_Run{str_run_number}.h5'
input_data = input_path / f'mfx101343025_Run{str_run_number}.h5'
synchrotron_path = cwd/'Synchrotron_data'
synchrotron_fq_path = synchrotron_path/f'{sample_name}_room.fq'
Background_path = Path().cwd() / 'kapton_background.txt'
output_path = cwd.parent/'MFX_hops'/'mfxl1044925'/'scratch'/'Output_files'

In [ ]:
with h5py.File(input_data, "r") as f:

    q = f['jungfrau']['pyfai_q'][:]  
    q = np.nanmean(q, axis=0)

    azav = f['jungfrau']['pyfai_azav'][:]                 
    Iq_shots = np.nanmean(azav, axis=1)                  

    delay_stage = f['scan']['mfx_lxt_fast2'][:].squeeze()
    delay_ps = delay_stage * 1e12

    laser = f['lightStatus']['laser'][:].squeeze()
    xray  = f['lightStatus']['xray'][:].squeeze()


xray_on = xray == 1

Iq_shots = Iq_shots[xray_on]
delay_ps = delay_ps[xray_on]
laser    = laser[xray_on]

laser_on  = laser == 1
laser_off = laser == 0

Iq_on_shots  = Iq_shots[laser_on]
Iq_off_shots = Iq_shots[laser_off]

delay_on  = delay_ps[laser_on]
delay_off = delay_ps[laser_off]


unique_delays = np.unique(delay_ps)

Iq_on = []
Iq_off = []
delay_clean = []

for d in unique_delays:
    mask_d = delay_ps == d
    
    laser_on  = mask_d & (laser == 1)
    laser_off = mask_d & (laser == 0)
    
    n_on  = np.sum(laser_on)
    n_off = np.sum(laser_off)
    
    # Only keep delay if BOTH exist
    if n_on > 0 and n_off > 0:
        Iq_on.append(np.nanmean(Iq_shots[laser_on], axis=0))
        Iq_off.append(np.nanmean(Iq_shots[laser_off], axis=0))
        delay_clean.append(d)

Iq_on = np.array(Iq_on)
Iq_off = np.array(Iq_off)
delay_clean = np.array(delay_clean)

sort_idx = np.argsort(delay_clean)
delay = delay_clean[sort_idx]
on = Iq_on[sort_idx]
off = Iq_off[sort_idx]
target_key = delay[target_id]
synchrotron_fq_load = pd.read_csv(synchrotron_fq_path,delimiter=' ',skiprows=26,index_col=0)
q_synchrotron = np.array(synchrotron_fq_load.index.tolist(), dtype=float) 
fq_synchrotron = synchrotron_fq_load.iloc[:, 0].values  

In [ ]:
def build_delay_dict(delays,delay_time,q,on,off,q_min=None,q_max=None):
    diff = on-off
    if q_min is not None:
        qmin_idx = find_nearest(q,q_min)
    else:
        qmin_idx = 0
    if q_max is not None:
        qmax_idx = find_nearest(q,q_max)
    else:
        qmax_idx = -1
    l1_diff = np.sum(np.abs(diff[qmin_idx:qmax_idx]))
    l2_diff = np.sum(diff[qmin_idx:qmax_idx]**2)
    diff_int = np.sum(diff[qmin_idx:qmax_idx])
    i_sum_off = np.sum(off[qmin_idx:qmax_idx])
    i_sum_on = np.sum(on[qmin_idx:qmax_idx])
    delays.update({delay_time : [
        q,
        on,
        off,
        diff,
        l1_diff,
        i_sum_off,
        i_sum_on,
        l2_diff,
        diff_int
    ]})
    return delays


def plot_function(delays):
    fig, (ax0,ax1,ax2,ax3,ax4, ax5) = plt.subplots(6,1,figsize=(8, 16))
    keys = [key for key in delays.keys()]
    delay_times_l1 = [delay[4] for delay in delays.values()]
    delay_times_off = [delay[5] for delay in delays.values()]
    delay_times_on = [delay[6] for delay in delays.values()]
    delay_times_l2 = [delay[7] for delay in delays.values()]
    delay_times_int = [delay[8] for delay in delays.values()]
    cmap = matplotlib.colormaps['viridis']
    colors = [cmap(i) for i in np.linspace(0, 1, len(keys))]
    key_to_color_idx = {key: i for i, key in enumerate(keys)}
    for key, delay in delays.items():
        if key == time_away_t0:
            on_plot = delay[1]
            off_plot = delay[2]
        color = colors[key_to_color_idx[key]]
        ax0.plot(delay[0],delay[1],label=key,color=color)
        ax1.plot(delay[0],delay[2],label=key,color=color) 
        ax2.plot(delay[0],delay[3],label=key,color=color)
        if q_min is not None:
            ax2.axvline(x=q_min,color='red')
        if q_max is not None:
            ax2.axvline(x=q_max,color='red')
            
    q_vals = list(delays.values())[0][0]
    on_minus_off_matrix = np.array([delay[3] for delay in delays.values()])
    delay_times = np.array([key for key in delays.keys()])
    sort_idx = np.argsort(delay_times)
    on_minus_off_matrix = on_minus_off_matrix[sort_idx]
    delay_times = delay_times[sort_idx]
    extent = [q_vals[0], q_vals[-1], delay_times[0], delay_times[-1]]
    ax3.imshow(on_minus_off_matrix, aspect='auto', extent=extent, origin='lower', cmap='viridis')
    ax3.invert_yaxis()
    if q_min is not None:
        ax3.axvline(x=q_min,color='red')
    if q_max is not None:
        ax3.axvline(x=q_max,color='red')
    ax3.set_xlabel("Q [1/A]")
    ax3.set_ylabel("Time delay (ps)")
    ax3.set_title("On - Off")
    ax4.plot(keys,delay_times_off,marker='o', linestyle='-',label='off')
    ax4.plot(keys,delay_times_on,marker='o', linestyle='-',label='on')
    ax5.plot(keys,np.sqrt(delay_times_l2),marker='o', linestyle='-',label='diff')
    ax5.plot(keys,delay_times_int,marker='o', linestyle='-',label='diff')
    ax0.set_xlabel('Q [1/A]')
    ax0.set_ylabel('Pump On Intensity [a.u.]')
    ax1.set_xlabel('Q [1/A]')
    ax1.set_ylabel('Pump Off Intensity [a.u.]')
    ax2.set_xlabel('Q [1/A]')
    ax2.set_ylabel('On-Off Intensity [a.u.]')
    ax4.set_xlabel('Time delay (ps)')
    ax4.set_ylabel('Sum intensities')
    ax5.set_xlabel('Time delay (ps)')
    ax5.set_ylabel('RMS')
    ax4.legend()
    ax0.set_title(f'sample = {sample_name}, run = {run_number}, qmin = {q_min}, qmax = {q_max}')
    ax1.set_title(f'run = {run_number}')
    ax2.set_title(f'I(q) On - I(q) Off run = {run_number}')
    ax5.set_title(f'Figure of Merit run = {run_number}, q_min = {q_min}, q_max = {q_max}')
    plt.tight_layout()
    plt.show()

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
raw_delays = {}
for i, step in enumerate(delay): 
    raw_delays = build_delay_dict(
        raw_delays,
        step,
        q,
        on[i],
        off[i],
        q_min,
        q_max
    )

morph_delays = {}
target = raw_delays[target_key]
for delay_t, data in raw_delays.items():

    x = data[0]          # q-grid
    y_on = data[1]       # ON signal
    y_off = data[2]      # OFF signal

    morph_on_table  = np.column_stack([x, y_on])
    morph_off_table = np.column_stack([x, y_off])
    target_table    = np.column_stack([x, target[1]])

    # First morph to target to get parameters
    info_on, _ = morph_arrays(
        morph_on_table,
        target_table,
        **params
    )

    info_off, _ = morph_arrays(
        morph_off_table,
        target_table,
        **params
    )

    # Then apply parameters without refining
    _, table_on_full = morph_arrays(
        morph_on_table,
        target_table,
        scale=info_on.get("scale", None),
        stretch=info_on.get("stretch", None),
        smear=info_on.get("smear", None),
        apply=True
    )

    _, table_off_full = morph_arrays(
        morph_off_table,
        target_table,
        scale=info_off.get("scale", None),
        stretch=info_off.get("stretch", None),
        smear=info_off.get("smear", None),
        apply=True
    )

    on_morph  = table_on_full[:, 1]
    off_morph = table_off_full[:, 1]


    morph_delays = build_delay_dict(
        morph_delays,
        delay_t,
        x,
        on_morph,
        off_morph,
        q_min,
        q_max
    )
    
if points_away_t0_plot_on_off is not None: 
    t0_index = find_nearest(delay,0)
    time_away_t0_index_plot = t0_index + points_away_t0_plot_on_off
    time_away_t0 = delay[time_away_t0_index_plot]

In [ ]:
plot_function(morph_delays)

In [ ]:
def pdfgetter_function(x, y, rpoly, qmin, bgscale):
    """
    Transform input I(Q) to F(Q) using PDFGetter with background correction.
    Returns (x, F(Q)).
    """
    cfg = PDFConfig()
    cfg.composition = sample_composition
    cfg.rpoly = rpoly
    cfg.qmin = qmin
    cfg.qmax = 20
    cfg.qmaxinst = 20
    cfg.dataformat = 'QA'
    cfg.mode = 'xray'

    background_path = Background_path
    if background_path.exists():
        bg_data = np.loadtxt(background_path)
        if bg_data.ndim == 2 and bg_data.shape[1] == 2:
            bg_interp = np.interp(x, bg_data[:, 0], bg_data[:, 1])
        else:
            bg_interp = bg_data  # already on same grid
        cfg.background = bg_interp
        cfg.bgscale = bgscale
    else:
        print(f"Background file not found: {background_path}")
        cfg.background = None
        cfg.bgscale = 0.0

    # Run PDFGetter
    pg = PDFGetter(cfg)
    _, _ = pg(x=x, y=y)
    q_out, fq_out = pg.fq

    fq_interp = np.interp(x, q_out, fq_out)
    return x, fq_interp

In [ ]:
reference_delay = delay[target_id]   

x_morph = morph_delays[reference_delay][0]
y_morph = morph_delays[reference_delay][2]
x_target = q_synchrotron
y_target = fq_synchrotron

morph_table = np.column_stack([x_morph, y_morph])
target_table = np.column_stack([x_target, y_target])

morph_info, morph_result = morph_arrays(
    morph_table,
    target_table,
    funcxy=(pdfgetter_function, initial_guesses),
    scale=1,
    squeeze=squeeze,
    xmin=fit_qmin,
    xmax=fit_qmax
)

fitted_funcxy = morph_info['funcxy']
scale_opt = float(morph_info['scale'])
squeeze_opt = [float(v) for v in morph_info['squeeze'].values()]

In [ ]:
def compute_gr(q, fq, r_min=0, r_max=30, npoints=1000):
    r = np.linspace(r_min, r_max, npoints)
    qr = np.outer(q, r)
    integrand = fq[:, None] * np.sin(qr)
    gr = (2/np.pi) * np.trapezoid(integrand, q, axis=0)
    return r, gr

def plot_reference_comparison(q_target, fq_target,
                              q_morph, fq_morph,
                              r_min=0, r_max=30,
                              r_min_fom=None, r_max_fom=None):

    r, gr_target = compute_gr(q_target, fq_target)
    r, gr_morph = compute_gr(q_morph, fq_morph)

    fig, (ax0, ax1) = plt.subplots(2,1, figsize=(8,6))

    ax0.plot(q_target, fq_target, linestyle='--', label="Synchrotron F(Q)")
    ax0.plot(q_morph, fq_morph, label="XFEL F(Q)")
    ax0.set_xlabel("Q (1/Å)")
    ax0.set_ylabel("F(Q)")
    ax0.set_title("Reference: F(Q)")
    ax0.legend()

    ax1.plot(r, gr_target, label="G(r) Synchrotron", color='orange')
    ax1.plot(r, gr_morph, label="G(r) XFEL", color='black')

    if r_min_fom is not None:
        ax1.axvline(r_min_fom, color='red')
    if r_max_fom is not None:
        ax1.axvline(r_max_fom, color='red')

    ax1.set_xlabel("r (Å)")
    ax1.set_ylabel("G(r)")
    ax1.legend()

    plt.tight_layout()
    plt.show()


def plot_gr_function(gr_delay_dict,
                     sample_name,
                     run_number,
                     r_min_fom=None,
                     r_max_fom=None):

    fig, (ax0, ax1, ax2, ax3, ax4) = plt.subplots(5,1, figsize=(8,14))

    delay_keys = sorted(gr_delay_dict.keys())
    cmap = matplotlib.colormaps['viridis']
    norm = plt.Normalize(min(delay_keys), max(delay_keys))

    for delay_t in delay_keys:
        pdata = gr_delay_dict[delay_t]
        color = cmap(norm(delay_t))

        ax0.plot(pdata["r"], pdata["gr_on"], color=color)
        ax1.plot(pdata["r"], pdata["gr_off"], color=color)
        ax2.plot(pdata["r"], pdata["diff_gr"], color=color)

    # ---- Integration window lines ----
    if r_min_fom is not None:
        ax2.axvline(r_min_fom, color='red')
    if r_max_fom is not None:
        ax2.axvline(r_max_fom, color='red')

    ax0.set_title(f"sample = {sample_name}, run = {run_number}")
    ax0.set_ylabel("G(r) ON")
    ax1.set_ylabel("G(r) OFF")
    ax2.set_ylabel("ΔG(r)")

    # ---- Delay metrics ----
    delay_times = delay_keys
    sum_on = [gr_delay_dict[d]["sum_gr_on"] for d in delay_times]
    sum_off = [gr_delay_dict[d]["sum_gr_off"] for d in delay_times]
    RMS = [gr_delay_dict[d]["RMS"] for d in delay_times]
    diff_int = [gr_delay_dict[d]["diff_int"] for d in delay_times]

    ax3.plot(delay_times, sum_on, marker='o', label='Pump ON')
    ax3.plot(delay_times, sum_off, marker='o', label='Pump OFF')
    ax3.set_ylabel("Integrated G(r)")
    ax3.legend(frameon=False)

    ax4.plot(delay_times, RMS, marker='o', label='RMS')
    ax4.plot(delay_times, diff_int, marker='o', label='Integral ΔG(r)')
    ax4.set_xlabel("Delay time")
    ax4.set_ylabel("Metric")
    ax4.legend(frameon=False)

    plt.tight_layout()
    plt.show()



In [ ]:
gr_delay_dict = {}

for delay_t in delay:

    q_iq  = morph_delays[delay_t][0]
    on_iq = morph_delays[delay_t][1]
    off_iq = morph_delays[delay_t][2]

    table_on  = np.column_stack([q_iq, on_iq])
    table_off = np.column_stack([q_iq, off_iq])

    target_dummy = table_on  # only to preserve grid

    _, fq_on_table = morph_arrays(
        table_on,
        target_dummy,
        funcxy=(pdfgetter_function, fitted_funcxy),
        scale=scale_opt,
        squeeze=squeeze_opt,
        xmin=fit_qmin,
        xmax=fit_qmax,
        apply=True
    )

    _, fq_off_table = morph_arrays(
        table_off,
        target_dummy,
        funcxy=(pdfgetter_function, fitted_funcxy),
        scale=scale_opt,
        squeeze=squeeze_opt,
        xmin=fit_qmin,
        xmax=fit_qmax,
        apply=True
    )

    q_final = fq_on_table[:,0]
    fq_on   = fq_on_table[:,1]
    fq_off  = fq_off_table[:,1]

    r, gr_on  = compute_gr(q_final, fq_on)
    r, gr_off = compute_gr(q_final, fq_off)

    diff = gr_on - gr_off

    rmin_idx = np.abs(r - r_min_fom).argmin()
    rmax_idx = np.abs(r - r_max_fom).argmin()

    gr_delay_dict[delay_t] = {
        "r": r,
        "gr_on": gr_on,
        "gr_off": gr_off,
        "diff_gr": diff,
        "RMS": np.sqrt(np.sum(diff[rmin_idx:rmax_idx]**2)),
        "diff_int": np.sum(diff[rmin_idx:rmax_idx]),
        "sum_gr_on": np.sum(gr_on[rmin_idx:rmax_idx]),
        "sum_gr_off": np.sum(gr_off[rmin_idx:rmax_idx]),
    }


In [ ]:
plot_reference_comparison(
    q_target=q_synchrotron,
    fq_target=fq_synchrotron,
    q_morph=morph_result[:, 0],
    fq_morph=morph_result[:, 1],
    r_min=0,
    r_max=30,
)

In [ ]:
plot_gr_function(
    gr_delay_dict,
    sample_name=sample_name,
    run_number=run_number,
    r_min_fom=r_min_fom,
    r_max_fom=r_max_fom
)